In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
# Create DataFrame
df = pd.read_csv('../data/FSA_DS_Men2019.csv')
df = df[['Rider Name', 'Price', 'Score 2019']]
df['slot'] = 1
df['24+'] = (df['Price'] >= 24).astype(int)
df['18+'] = (df['Price'] >= 18).astype(int)
df

,Rider Name,Price,Score 2019,slot,24+,18+
0,Primož Roglič,28,4070,1,1,1
1,Alejandro Valverde,34,2910,1,1,1
2,Egan Bernal,24,2752,1,1,1
3,Julian Alaphilippe,30,2601,1,1,1
4,Pascal Ackermann,20,2462,1,0,1
...,...,...,...,...,...,...
1373,Edoardo Zardini,1,0,1,0,0
1374,Eugert Zhupa,1,0,1,0,0
1375,Kamil Zielinski,1,0,1,0,0
1376,Maikel Zijlaard,1,0,1,0,0


In [3]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [list(df['Price']),
                                 list(df['slot']),
                                 list(df['24+']),
                                 list(df['18+'])]
    data['bounds'] = [150, 25, 1, 3]
    data['obj_coeffs'] = list(df['Score 2019'])
    data['num_vars'] = len(df)
    data['num_constraints'] = len(data['bounds'])
    return data

data = create_data_model()
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver('simple_mip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, 1, df.loc[j, 'Rider Name'])
print('Number of variables =', solver.NumVariables())

for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
print('Number of constraints =', solver.NumConstraints())
# In Python, you can also set the constraints as follows.
# for i in range(data['num_constraints']):
#    constraint_expr = \
# [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#    solver.Add(sum(constraint_expr) <= data['bounds'][i])

objective = solver.Objective()
for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
objective.SetMaximization()
# In Python, you can also set the objective as follows.
# obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
# solver.Maximize(solver.Sum(obj_expr))

status = solver.Solve()

selected_riders = []

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
        if x[j].solution_value():
            selected_riders.append(j)
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')
    
df.loc[selected_riders]

Number of variables = 1378
Number of constraints = 4
Objective value = 35876.0

Problem solved in 116.000000 milliseconds
Problem solved in 88 iterations
Problem solved in 22 branch-and-bound nodes


,Rider Name,Price,Score 2019,slot,24+,18+
0,Primož Roglič,28,4070,1,1,1
4,Pascal Ackermann,20,2462,1,0,1
5,Jakob Fuglsang,8,2421,1,0,0
10,Mathieu van der Poel,8,1778,1,0,0
11,Tadej Pogacar,1,1736,1,0,0
12,Matteo Trentin,10,1713,1,0,0
13,Bauke Mollema,12,1690,1,0,0
17,Yves Lampaert,10,1562,1,0,0
18,Alexey Lutsenko,4,1525,1,0,0
19,Sam Bennett,10,1517,1,0,0


In [4]:
df.loc[selected_riders].sum()

Rider Name    Primož RogličPascal AckermannJakob FuglsangMat...
Price                                                       150
Score 2019                                                35876
slot                                                         25
24+                                                           1
18+                                                           2
dtype: object

# Women

In [5]:
# Create DataFrame
df = pd.read_csv('../data/FSA_DS_Women2019.csv')
df = df[['Rider Name', 'Price', 'Score 2019']]
df['slot'] = 1
df['20+'] = df['Price'].apply(lambda x: x if x >= 20 else 0)
df

,Rider Name,Price,Score 2019,slot,20+
0,Annemiek van Vleuten,52,4260,1,52
1,Marianne Vos,36,4020,1,36
2,Lorena Wiebes,18,3373,1,0
3,Marta Bastianelli,22,3348,1,22
4,Anna van der Breggen,52,2956,1,52
...,...,...,...,...,...
562,Tara Whitten,1,0,1,0
563,Sophie Wright,1,0,1,0
564,Chiara Zanettin,1,0,1,0
565,Sasa Zavbi Kunager,1,0,1,0


In [6]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['constraint_coeffs'] = [list(df['Price']),
                                 list(df['slot']),
                                 list(df['20+'])]
    data['bounds'] = [150, 15, 100]
    data['obj_coeffs'] = list(df['Score 2019'])
    data['num_vars'] = len(df)
    data['num_constraints'] = len(data['bounds'])
    return data

data = create_data_model()
# Create the mip solver with the CBC backend.
solver = pywraplp.Solver('simple_mip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
infinity = solver.infinity()
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, 1, df.loc[j, 'Rider Name'])
print('Number of variables =', solver.NumVariables())

for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
print('Number of constraints =', solver.NumConstraints())
# In Python, you can also set the constraints as follows.
# for i in range(data['num_constraints']):
#    constraint_expr = \
# [data['constraint_coeffs'][i][j] * x[j] for j in range(data['num_vars'])]
#    solver.Add(sum(constraint_expr) <= data['bounds'][i])

objective = solver.Objective()
for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
objective.SetMaximization()
# In Python, you can also set the objective as follows.
# obj_expr = [data['obj_coeffs'][j] * x[j] for j in range(data['num_vars'])]
# solver.Maximize(solver.Sum(obj_expr))

status = solver.Solve()

selected_riders = []

if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for j in range(data['num_vars']):
        if x[j].solution_value():
            selected_riders.append(j)
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')
    
df.loc[selected_riders]

Number of variables = 567
Number of constraints = 3
Objective value = 27706.0

Problem solved in 54.000000 milliseconds
Problem solved in 6 iterations
Problem solved in 0 branch-and-bound nodes


,Rider Name,Price,Score 2019,slot,20+
1,Marianne Vos,36,4020,1,36
2,Lorena Wiebes,18,3373,1,0
3,Marta Bastianelli,22,3348,1,22
5,Katarzyna Niewiadoma,26,2611,1,26
7,Soraya Paladin,4,2141,1,0
10,Lisa Klein,10,1962,1,0
13,Demi Vollering,1,1805,1,0
14,Lotte Kopecky,6,1660,1,0
20,Sofie De Vuyst,4,1290,1,0
23,Chloe Dygert-Owen,6,1220,1,0


In [7]:
df.loc[selected_riders].sum()

Rider Name    Marianne VosLorena WiebesMarta BastianelliKata...
Price                                                       150
Score 2019                                                27706
slot                                                         15
20+                                                          84
dtype: object